# Fastai Tutorial for VinBigDate Chest Xray Abnormalities Detection

## With Weighted Bbox fusion

In [ ]:
!pip uninstall fastai -y

In [ ]:
import os,sys
sys.path.append('../input/fastaiv1')
!pip install -q object-detection-fastai

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
import os
from tqdm.notebook import tqdm
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.image as immg
from torchvision.models.detection.rpn import AnchorGenerator
from sklearn.model_selection import StratifiedKFold,KFold

from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric

In [ ]:
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
sns.set_style('darkgrid')

In [ ]:
df = pd.read_csv('../input/vinbigdata-weighted-bbox-fusion/weighted_box_fused_train_vinBigData.csv')
img_dim = pd.read_csv('../input/vinbigdata-resized-image-512/train_meta.csv')
tr_img_dir = Path('../input/vinbigdata-resized-image-512/train')
ts_img_dir = Path('../input/vinbigdata-resized-image-512/test')

In [ ]:
tr_df = df.merge(img_dim,on='image_id',how='left')

In [ ]:
tr_df.head(10)

In [ ]:
tr_df1 = tr_df[(tr_df['class_id']!=14)&(tr_df['class_id']!=4)&(tr_df['class_id']!=2)&(tr_df['class_id']!=9)].copy()

## Rescaling bounding boxes according to resized images

In [ ]:
tr_df1['x_min'] = tr_df1['x_min']*512/tr_df['dim1']
tr_df1['x_max'] = tr_df1['x_max']*512/tr_df['dim1']
tr_df1['y_min'] = tr_df1['y_min']*512/tr_df['dim0']
tr_df1['y_max'] = tr_df1['y_max']*512/tr_df['dim0']

In [ ]:
tr_df1.head()

In [ ]:
tr_df1.class_id.value_counts()

In [ ]:
unq_id = tr_df1.image_id.unique()

In [ ]:
df_grp = tr_df1.groupby(['image_id'])

In [ ]:
one_hot = {}
for i in tqdm(unq_id):
    l = np.zeros(14)
    temp = df_grp.get_group(i)
    for j in temp.class_id.unique():
        l[j] = 1
    one_hot[i] = l

In [ ]:
df_grp.get_group('9a5094b2563a1ef3ff50dc5c7ff71345')

In [ ]:
fold_df = pd.DataFrame.from_dict(one_hot,orient='index');
fold_df.reset_index(inplace=True)
fold_df.columns = ['image_id']+fold_df.columns.tolist()[1:]

In [ ]:
sys.path.append('../input/multistartifiedkfold')

## MultiStratified KFOLD

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
fold_df['kfold'] = -1
fold_df = fold_df.sample(frac=1.0).reset_index(drop=True)
y = fold_df[fold_df.columns.tolist()[1:]].values
kf = MultilabelStratifiedKFold(n_splits=5)
for fld, (trn_,val_) in enumerate(kf.split(X=fold_df,y=y)):
    fold_df.loc[val_,'kfold'] = fld

In [ ]:
fold_df.head()

In [ ]:
b_fea = ['x_min', 'y_min', 'x_max', 'y_max']

## Sample Check

In [ ]:
name = '0c7a38f293d5f5e4846aa4ca6db4daf1'
loc = '../input/vinbigdata-resized-image-512/train/'+name+'.png'
aaa = df_grp.get_group(name)
bbx = aaa.loc[:,b_fea]
img = immg.imread(loc)
fig,ax = plt.subplots(figsize=(18,10))
ax.imshow(img,cmap='binary')
for i in range(len(bbx)):
    box = bbx.iloc[i].values
    x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none',)
    ax.text(*box[:2], aaa['class_id'].iloc[i], verticalalignment='top', color='white', fontsize=12, weight='bold')
    ax.add_patch(rect)
plt.show()

## Get lblbox

In [ ]:
def get_lbl_img(train):
    chest2bbox = {}
    grp = train.image_id.unique()
    tr_gr = train.groupby(['image_id'])
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(grp))):
        name = str(grp[i])+'.png'
        bbox = []
        lbls = []
        temp_b = []
        temp = tr_gr.get_group(grp[i])
        tt = temp.loc[:, (['class_id','x_min', 'y_min', 'x_max', 'y_max'])].values
        for j in range(len(temp)):
            lbls.append(tt[j][0].astype(int))
            b = list(np.round(tt[j][1:]))   # x,y, width, height
            # Currently our coordinates are x,w,l,h and we want x1,y1,x2,y2
            # To convert it, we need to add our width and height to the respective x and y.
            t1 = [b[1],b[0],b[3],b[2]]

            temp_b.append(t1)
        bbox.append(temp_b)
        bbox.append(lbls)
        chest2bbox[name] = bbox
    return chest2bbox

In [ ]:
chest2bbox = get_lbl_img(tr_df1)

In [ ]:
chest2bbox['0c7a38f293d5f5e4846aa4ca6db4daf1.png']

In [ ]:
get_y_func = lambda o: chest2bbox[Path(o).name] 

In [ ]:
tr = tr_df1.image_id.value_counts()
tr = pd.DataFrame({'image_id':tr.index,'image_count':tr.values})
tr = tr.sample(frac=1.,random_state=2020).reset_index(drop=True)

In [ ]:
fold_df.to_csv('kfold_train.csv')

In [ ]:
FOLD = 2

In [ ]:
trn_idx,val_idx = fold_df[fold_df['kfold']!=FOLD].index,fold_df[fold_df['kfold']==FOLD].index

In [ ]:
len(trn_idx),len(val_idx)

In [ ]:
data = (ObjectItemList.from_df(fold_df,path='../input/vinbigdata-resized-image-512', folder = 'train' ,cols='image_id',suffix='.png')
        #Where are the images? ->
        .split_by_idxs(train_idx=trn_idx, valid_idx=val_idx)                         
        #How to split in train/valid? -> randomly with the default 20% in valid
        .label_from_func(get_y_func)
        #How to find the labels? -> use get_y_func on the file name of the data
        .transform(size=512,resize_method=ResizeMethod.SQUISH)
        #.add_test(ts)
        #Data augmentation? -> Standard transforms; also transform the label images
        .databunch(bs=16, collate_fn=bb_pad_collate))

In [ ]:
data.show_batch( 1, figsize = (15,10))

In [ ]:
len(data.train_ds),len(data.valid_ds)

In [ ]:
data.classes,data.c

In [ ]:
size = 512

In [ ]:
anchors = create_anchors(sizes=[(32,32),(16,16),(8,8),(4,4)], ratios=[0.5, 1.0, 2.0], scales=[0.25, 0.65, 1.25, 2.5, 3.5, 4.5])

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
ax.imshow(image2np(data.valid_ds[0][0].data))

for i, bbox in enumerate(anchors[:18]):
    bb = bbox.numpy()
    x = (bb[0] + 1) * size / 2 
    y = (bb[1] + 1) * size / 2 
    w = bb[2] * size / 2
    h = bb[3] * size / 2
    
    rect = [x,y,w,h]
    draw_rect(ax,rect)

In [ ]:
len(anchors)

In [ ]:
n_classes = data.train_ds.c

crit = RetinaNetFocalLoss(anchors)

encoder = create_body(models.resnet34, True, -2)

model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=18, sizes=[32,16,8,4], chs=32, final_bias = -4., n_conv = 2)

In [ ]:
voc = PascalVOCMetric(anchors, size, [i for i in data.train_ds.y.classes[1:]])
learn = Learner(data,
                model, 
                loss_func=crit,
                callback_fns=[BBMetrics],
                metrics=[voc],
                model_dir = '/kaggle/working/')

In [ ]:
learn.split([model.encoder[6], model.c5top5]);
learn.freeze_to(-2)
#learn = learn.to_fp16()

In [ ]:
learn.fit_one_cycle(3, 1e-3 ,callbacks = [SaveModelCallback(learn, every ='improvement', monitor ='valid_loss', name ='best_model1',mode='min')])

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10,1e-3/2,callbacks = [SaveModelCallback(learn, every ='improvement', monitor ='valid_loss', name ='best_model2',mode='min')])

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.load('best_model2');

In [ ]:
show_results_side_by_side(learn, anchors, detect_thresh=0.4, nms_thresh=0.1, image_count=10)